<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/%ED%8A%B9%EC%A0%95_LLM_%EB%AA%A8%EB%8D%B8_(%EC%98%88_Llama_2%2C_Alpaca)%EC%9D%84_Foundation_%EB%AA%A8%EB%8D%B8%EB%A1%9C_%EC%84%A0%ED%83%9D%ED%95%98%EC%97%AC_%EB%8F%84%EB%A9%94%EC%9D%B8_%ED%8A%B9%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A1%9C_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

도메인 특화 파인튜닝 QA 파인튜닝
```
LLAMA2 Foundation 모델을 특정 도메인(ex 고객지원QA)에 맞게 최적화
instruction + question+answer -> instruction tuning
```
파인튜닝 전략
```
full fine-tuning : 모든 파라메터 학습 7B 모델기준 14GB
LoRA : 저차원 행렬로 파라메터 일부 학습 (0.1~1% 파라메터)
QLoRA : LoRA + 4비트 양자화
DPO : 선호도 데이터 기반 최적화, QA 직접적용 복잡
```
QLoRA 구조
```
  4-bit 양자화 : 가중치를 4비트로 압축 -> 메모리 절약(NF 4형식)
  중첩 양자화(Double Quantization) : 양자화 한것을 추가로 압축
  Paged Optimized : GPU메모리 부족시 CPU로 페이징 OOM방지(Out Of Memory)
```
LLAMA2 Foundation 모델
```
Meta AI 개발, 2023 7월, 비상업적 무료,
LLaMA 2-7B,LLaMA 2-13B,LLaMA 2-70B
모델구성 : Transformer decoder-only (GPT 계열)
토크나이져 : SentencePice 기반 BPE 토크나이져
PEFT, QLoRA  파인튜닝에 유
```
SentencePice
```
구글에서 만든 토크나이져, 공백단위가 아니라 문자단위 기반
공백무시기능,다국어 적합, 서브워드 기반, 전처리보다는 원시 텍스트 입력가능(hello world!)
```
BPE(Byte Pair Encoding)
```
가장 많이 사용되는 서브워드 토크나이징 알고리즘
자주 나오는 문자쌍을 반복적으로 병합해서 단어를구
```
AutoModelForCausalLM
```
Causal Language Modeling(CLM) 태스크를 수행하는 모델을 자동으로 불러오는 허깅페이스 트랜스포머 클래스를 추상화한 클래스
Causal : 순방향 언어 모델링(GPT 스타일)
LM : Language Model
Auto : 모델 타입에 따라 알맞은 클래스를 자동으로 연결
어떤 모델? : gpt계열, LLaMA계열(llama, llama2)
트래스스포머 모델중에 Decode only계열가능
encoder-decode 모델 T5등 AutoModelForSeq2SeqLM사용
```

In [ ]:
!pip install --upgrade trl torch torchvision transformers bitsandbytes peft datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 21.3 MB/s eta 0:00:00


In [ ]:
# LLaMA2-7B 를 QLoRA로 로드하고 GPU 점검
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
import os

# gpu 체크
print(f'gpu available : {torch.cuda.is_available()}')
print(f'gpu count : {torch.cuda.device_count()}')
print(f'gpu name : {torch.cuda.get_device_name(0)}')
print(f'VRAM : {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f}GB')

# QLoRA 설정
bnb_config =  BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    )
# llama 계열 모델 로드
model_name = 'nvidia/Llama-3.1-Nemotron-Nano-4B-v1.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config,
                                             device_map='auto',
                                             )
tokenizer.pad_token_id = tokenizer.eos_token_id
# QA 형식 텍스트 입력
prompt = [
    {'role':'system','content':'detailed thinking off'},
    {'role':'user','content':'고객질문에 간단히 답변하세요: 배송지연시 어떻게 해야 하나요?'}
]
inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
                                      add_generation_prompt=True,return_tensors='pt').to('cuda')
outputs = model.generate(inputs, max_new_tokens=50,temperature=0.6,top_p=0.95)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

gpu available : True
gpu count : 1
gpu name : Tesla T4
VRAM : 14.74GB


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system

detailed thinking offuser

고객질문에 간단히 답변하세요: 배송지연시 어떻게 해야 하나요?assistant

고객질문에 대한 간단한 답변을 제공해드리겠습니다. 고객의 배송지연에 대한 질문은 다양한 경우가 있지만, 일반적으로 고객이 EXPECTED DELIVERY TIME (예상 배송 시간)에 불만하는 경우


QA데이터 셋(정제 및 프롬프트화) - json형식으로만들고 데이터셋형태로 로드

In [ ]:
import json
from datasets import Dataset
import pandas as pd

# QA 데이터셋 저장
# qa_data = [
#     {
#         "instruction": "고객 질문에 간단히 답변하세요",
#         "question": "배송 지연 시 어떻게 해야 하나요?",
#         "answer": "배송 지연 시 고객 지원팀에 연락해 추적 번호를 확인하세요."
#     },
#     {
#         "instruction": "고객 질문에 간단히 답변하세요",
#         "question": "환불 절차는 어떻게 되나요?",
#         "answer": "제품 반품 후 7일 이내 환불이 처리됩니다."
#     },
#     {
#         "instruction": "고객 질문에 간단히 답변하세요",
#         "question": "제품 보증 기간은 얼마나 되나요?",
#         "answer": "제품 보증 기간은 구매일로부터 1년입니다."
#     }
# ]

# # JSON 파일로 저장
# with open("customer_support_qa.json", "w", encoding="utf-8") as f:
#     json.dump(qa_data, f, ensure_ascii=False, indent=2)

# JSON → Dataset 변환
df = pd.read_json("/content/skin_custormer.json")
dataset = Dataset.from_pandas(df)

# 데이터 전처리: 결측값 및 데이터 검증
def preprocess_function(examples):
    for key in ["instruction", "question", "answer"]:
        if not all(isinstance(x, str) for x in examples[key]):
            raise ValueError(f"Non-string value found in {key}")
    return examples

dataset = dataset.map(preprocess_function, batched=True)
print("Dataset Info:", dataset)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset Info: Dataset({
    features: ['instruction', 'question', 'answer'],
    num_rows: 10
})


In [ ]:
from transformers import AutoTokenizer
from typing import Dict, List

# 토크나이저
model_name = "nvidia/Llama-3.1-Nemotron-Nano-4B-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

# PromptTemplate 클래스
class PromptTemplate:
    def __init__(self):
        self.system_prompt = "고객 질문에 간단히 답변하세요"

    def format(self, question: str, answer: str = "") -> List[Dict]:
        # Nemotron-Nano의 apply_chat_template 형식
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": question}
        ]
        if answer:
            messages.append({"role": "assistant", "content": answer})
        return messages

# 샘플 프롬프트 생성
qa_template = PromptTemplate()
sample = dataset[0]
chat_prompt = qa_template.format(
    question=sample["question"],
    answer=sample["answer"]
)
print("Chat Prompt:", chat_prompt)

# 토크나이저로 프롬프트 변환
tokenized_prompt = tokenizer.apply_chat_template(
    chat_prompt,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)
print("Tokenized Prompt (first 10 tokens):", tokenized_prompt[0][:10])

Chat Prompt: [{'role': 'system', 'content': '고객 질문에 간단히 답변하세요'}, {'role': 'user', 'content': '성인 여드름의 원인은 무엇인가요?'}, {'role': 'assistant', 'content': '성인 여드름은 호르몬 변화, 스트레스, 특정 약물 복용, 화장품 사용 등이 원인이 될 수 있습니다.'}]
Tokenized Prompt (first 10 tokens): tensor([128000, 128006,   9125, 128007,    271,  35495, 108583, 109760,  19954,
        105131])


In [ ]:
import torch

def tokenize_function(examples, tokenizer, max_length=512):
    # 배치 단위 프롬프트 생성
    prompts = [
        qa_template.format(question=q, answer=a)
        for q, a in zip(examples["question"], examples["answer"])
    ]

    # apply_chat_template으로 토크나이징
    tokenized = [
        tokenizer.apply_chat_template(
            prompt,
            tokenize=True,
            add_generation_prompt=False,
            return_tensors="pt"
        )[0]
        for prompt in prompts
    ]

    # 패딩 및 길이 제한
    input_ids = torch.nn.utils.rnn.pad_sequence(
        [t for t in tokenized],
        batch_first=True,
        padding_value=tokenizer.pad_token_id
    )
    input_ids = input_ids[:, :max_length]

    # 마스크 및 레이블 생성
    attention_mask = (input_ids != tokenizer.pad_token_id).long()
    labels = input_ids.clone()

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# 데이터셋 토크나이징
tokenized_dataset = dataset.map(
    lambda x: tokenize_function(x, tokenizer, max_length=512),
    batched=True,
    remove_columns=["instruction", "question", "answer"],
    num_proc=1  # 소규모 데이터셋이므로 num_proc=1
)

# 토큰 길이 점검
print("Sample tokenized input:", tokenized_dataset[0]["input_ids"][:10])
print("Sample length:", len(tokenized_dataset[0]["input_ids"]))

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Sample tokenized input: [128000, 128006, 9125, 128007, 271, 35495, 108583, 109760, 19954, 105131]
Sample length: 87


QLoRA -> Trainer 커스트마이징
```
PEFT QLoRA 설정
  전체파라메터의 0.1~1%사용 , loraconfig를 통해 target 모델 지정(ex q_proj, v_proj)
QLoRA : LoRA + 4bit 양자화  
SFTTrainer
  허깅페이스 trl 라이브러리
  하이퍼파라메터,학습손실, 평가단계 조정
학습전략
  batch size
  Gradient Checkpoint : 메모리절약, 중간계산과정을 저장
  LearningRate Scheduler : cosin 스케줄러 학습률을 점진적 감소    
```

In [ ]:
# QLoRA 설정
bnb_config =  BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    )
# llama 계열 모델 로드
model_name = 'nvidia/Llama-3.1-Nemotron-Nano-4B-v1.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 양자화 모델로드
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config,
                                             device_map='auto',
                                             )
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PEFT QLoRA 설정

In [ ]:
#모델의 어텐션 블럭을 확인
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Lin

In [ ]:
from peft import LoraConfig, get_peft_model
# LoRA 설정
lora_config = LoraConfig(
    target_modules=["q_proj", "v_proj"],
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
# 모델에 로라 적용
model = get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 5,767,168 || all params: 4,518,513,664 || trainable%: 0.1276


커스텀 SFTTrainer
  - trl.SFTTrainer로 학습설정, 손실로깅, 평가단계 커스터마이징

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import logging

In [ ]:
# llama 3 허깅페이스 로드
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 57, in main
    service.run()
  File "